# rough

In [2]:
import os
os.chdir('/content/drive/MyDrive/BTL_questionaire')
os.listdir()

['results', '.ipynb_checkpoints', 'model.ipynb', 'dataset_main.csv']

In [6]:
import pandas as pd

# read an excel file and convert
# into a dataframe object
df = pd.read_csv("dataset_main.csv")


In [7]:
df.head()

,BTL,QUESTION
0,1,Are there any scenarios or types of data where...
1,1,Are there any variations or balancing techniqu...
2,1,Are there scenarios where a binary search tree...
3,1,Are there scenarios where a non-self-balancing...
4,1,Can a binary search tree be empty?


In [16]:
df = df.drop_duplicates(subset=['QUESTION'])

In [17]:
df["BTL "].value_counts()

,count
BTL,
1,214
6,193
5,180
3,176
4,158
2,108


# Dataset Initializaiton

In [18]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [19]:
print("Training set shape:", train_df.shape)
print("Test set shape:", test_df.shape)

Training set shape: (823, 2)
Test set shape: (206, 2)


In [20]:
train_df.to_csv('train_df.csv')

In [21]:
test_df.to_csv('test_df.csv')

In [22]:
train_df = pd.read_csv('train_df.csv')
test_df = pd.read_csv('test_df.csv')

In [23]:
print(len(train_df))
print(len(test_df))

823
206


# SVM

In [24]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import make_pipeline

In [25]:
vectorizer = TfidfVectorizer()
svm_model = SVC(kernel='linear', random_state=42)


pipeline = make_pipeline(vectorizer, svm_model)


In [26]:
pipeline.fit(train_df['QUESTION'], train_df['BTL '])


Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('svc', SVC(kernel='linear', random_state=42))])

In [29]:
predictions = pipeline.predict(test_df['QUESTION'])

print("Accuracy:", accuracy_score(test_df['BTL '], predictions))
print("\nClassification Report:\n", classification_report(test_df['BTL '], predictions))


Accuracy: 0.7475728155339806

Classification Report:
               precision    recall  f1-score   support

           1       0.69      0.84      0.76        43
           2       0.77      0.77      0.77        22
           3       0.66      0.55      0.60        38
           4       0.79      0.67      0.72        39
           5       0.67      0.71      0.69        31
           6       0.94      0.97      0.96        33

    accuracy                           0.75       206
   macro avg       0.75      0.75      0.75       206
weighted avg       0.75      0.75      0.74       206



# BERT

In [ ]:
#!pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import load_dataset, Dataset
from transformers import DataCollatorWithPadding

In [ ]:
# Convert to Hugging Face Dataset format
train_dataset = load_dataset('csv', data_files='train_df.csv')
test_dataset = load_dataset('csv', data_files='test_df.csv')

train_dataset = train_dataset['train']
test_dataset = test_dataset['train']

In [ ]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)

# Tokenize the data
def tokenize_function(examples):
    return tokenizer(examples['QUESTION'], padding="max_length", truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set format to torch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'BTL '])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'BTL '])


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/480 [00:00<?, ? examples/s]

Map:   0%|          | 0/121 [00:00<?, ? examples/s]

In [ ]:
num_labels = 6  # For BTL levels 1 to 6
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
#this initialization takes a lot of ram

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

In [ ]:
results = trainer.evaluate()
print(results)

In [ ]:
def predict_btl(question):
    inputs = tokenizer(question, return_tensors="pt", padding=True, truncation=True, max_length=128)
    outputs = model(**inputs)
    predicted_label = torch.argmax(outputs.logits, dim=1).item() + 1  # Adding 1 to match BTL range 1-6
    return predicted_label

# Example
question = "What is the time complexity of quicksort?"
predicted_btl = predict_btl(question)
print(f"Predicted BTL Level: {predicted_btl}")
